In [17]:
using ITensors
using ITensors: position!
using ITensorTDVP
using KrylovKit
using Printf

using ProgressMeter
#ProgressMeter.ijulia_behavior(:append)


In [18]:
function measure_Sz(psi,n)
    psi = ITensors.orthogonalize(psi,n)
    sn = siteind(psi,n)
    Sz = scalar(dag(prime(psi[n],"Site"))*op("Sz",sn)*psi[n])
    return real(Sz)
end

measure_Sz (generic function with 1 method)

In [27]:
export tdvp!
using ITensors: position!

singlesite!(PH::ProjMPO) = (PH.nsite = 1)
twosite!(PH::ProjMPO) = (PH.nsite = 2)

struct TDVP2 end

"""
    tdvp!(psi,H::MPO,dt,tf; kwargs...)
Evolve the MPS `psi` up to time `tf` using the two-site time-dependent variational
principle as described in Ref. [1].
# Keyword arguments:
All keyword arguments controlling truncation which are accepted by ITensors.replaceBond!,
namely:
- `maxdim::Int`: If specified, keep only `maxdim` largest singular values after applying gate.
- `mindim::Int`: Minimal number of singular values to keep if truncation is performed according to
    value specified by `cutoff`.
- `cutoff::Float`: If specified, keep the minimal number of singular-values such that the discarded weight is
    smaller than `cutoff` (but bond dimension will be kept smaller than `maxdim`).
- `absoluteCutoff::Bool`: If `true` truncate all singular-values whose square is smaller than `cutoff`.
In addition the following keyword arguments are supported:
- `hermitian::Bool` (`true`) : whether the MPO `H` represents an Hermitian operator. This will be passed to the
    Krylov exponentiation routine (`KrylovKit.exponentiate`) which will in turn use a Lancosz algorithm in the
    case of an hermitian operator.
- `exp_tol::Float` (1e-14) : The error tolerance for `KrylovKit.exponentiate`.
    (note that default value was not optimized yet, so you might want to play around with it)
- `progress::Bool` (`true`) : If `true` a progress bar will be displayed
# References:
[1] Haegeman, J., Lubich, C., Oseledets, I., Vandereycken, B., & Verstraete, F. (2016).
Unifying time evolution and optimization with matrix product states. Physical Review B, 94(16).
https://doi.org/10.1103/PhysRevB.94.165116
"""
function tdvp!(psi,H::MPO,dt,tf; kwargs...)
    nsteps = Int(tf/dt)
    #cb = get(kwargs,:callback, NoTEvoCallback())
    hermitian = get(kwargs,:hermitian,true)
    exp_tol = get(kwargs,:exp_tol, 1e-14)
    krylovdim = get(kwargs,:krylovdim, 30 )
    maxiter = get(kwargs,:maxiter,100)
    normalize = get(kwargs,:normalize,true)

    pbar = get(kwargs,:progress, true) ? Progress(nsteps, desc="Evolving state... ") : nothing
    τ = 1im*dt
    imag(τ) == 0 && (τ = real(τ))

    N = length(psi)
    ITensors.orthogonalize!(psi,1)
    PH = ProjMPO(H)
    position!(PH,psi,1)
    for s in 1:nsteps
        stime = @elapsed begin
        for (b,ha) in sweepnext(N)
            #evolve with two-site Hamiltonian
            twosite!(PH)
            ITensors.position!(PH,psi,b)
            wf = psi[b]*psi[b+1]
            wf, info = exponentiate(PH, -τ/2, wf; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
            dir = ha==1 ? "left" : "right"
            info.converged==0 && throw("exponentiate did not converge")
            spec = replacebond!(psi,b,wf;normalize=normalize, ortho = dir, kwargs... )
            # normalize && ( psi[dir=="left" ? b+1 : b] /= sqrt(sum(eigs(spec))) )
            #=
            apply!(cb,psi; t=s*dt,
                   bond=b,
                   sweepend= ha==2,
                   sweepdir= ha==1 ? "right" : "left",
                   spec=spec,
                   alg=TDVP2())
            =#

            # evolve with single-site Hamiltonian backward in time.
            # In the case of imaginary time-evolution this step
            # is not necessary (see Ref. [1])
            i = ha==1 ? b+1 : b
            if 1<i<N && !(dt isa Complex)
                singlesite!(PH)
                ITensors.position!(PH,psi,i)
                psi[i], info = exponentiate(PH,τ/2,psi[i]; ishermitian=hermitian, tol=exp_tol, krylovdim=krylovdim,
                                            maxiter=maxiter)
                info.converged==0 && throw("exponentiate did not converge")
            elseif i==1 && dt isa Complex
                # TODO not sure if this is necessary anymore
                psi[i] /= sqrt(real(scalar(dag(psi[i])*psi[i])))
            end

        end
        end
        !isnothing(pbar) && ProgressMeter.next!(pbar, showvalues=[("t", dt*s),
                                                                  ("dt step time", round(stime,digits=3)),
                                                                  ("Max bond-dim", maxlinkdim(psi))])
        #checkdone!(cb) && break
    end
end

tdvp!

In [28]:
N = 15

s = siteinds("S=1/2", N)

os = OpSum()
for j in 1:(N - 1)
    os += 0.5, "S+", j, "S-", j + 1
    os += 0.5, "S-", j, "S+", j + 1
    os += "Sz", j, "Sz", j + 1
end

H = MPO(os, s)
#ψ = productMPS(s, n -> isodd(n) ? "Up" : "Dn")
ψ=randomMPS(s; linkdims=5)
#psi = productMPS(s, n -> isodd(n) ? "Up" : "Dn")


for j=1:N
    println("site : ",j," ",measure_Sz(ψ,j))
end


site : 1 0.27902255077953314
site : 2 0.11817057880196195
site : 3 -0.1699914253375467
site : 4 -0.11250046529889368
site : 5 -0.06599371885151037
site : 6 -0.03204545340132578
site : 7 0.05619019903028113
site : 8 0.03549310258727675
site : 9 -0.08882634724034837
site : 10 -0.05604017443275978
site : 11 -0.07076978946509876
site : 12 0.07593901185729657
site : 13 -0.024272228754986722
site : 14 0.07978853217244722
site : 15 0.01457790322514336


In [29]:
ψ1 = deepcopy(ψ)
#2-site TDVP
tdvp!(ψ1,H::MPO,0.05,5)

#Calculate magnetisation at all sites.
for j=1:N
    println("site : ",j," ",measure_Sz(ψ1,j))
end

site : 1 0.27902255077953314
site : 2 0.11817057880196195
site : 3 -0.1699914253375467
site : 4 -0.11250046529889368
site : 5 -0.06599371885151037
site : 6 -0.03204545340132578
site : 7 0.05619019903028113
site : 8 0.03549310258727675
site : 9 -0.08882634724034837
site : 10 -0.05604017443275978
site : 11 -0.07076978946509876
site : 12 0.07593901185729657
site : 13 -0.024272228754986722
site : 14 0.07978853217244722
site : 15 0.01457790322514336

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/sandipan/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Evolving state... 100%|█████████████████████████████████| Time: 0:05:07
  t:             5.0
  dt step time:  3.201
  Max bond-dim:  128


site : 1 0.04278362893220084
site : 2 0.031842079281770685
site : 3 0.04903011341452134
site : 4 0.007806360650385813
site : 5 -0.025089162869784746
site : 6 -0.01881147052875
site : 7 -0.015809634361544384
site : 8 -0.04619067504831747
site : 9 -0.014559288336780416
site : 10 -0.025351996781418185
site : 11 -0.008724494388648764
site : 12 0.007830775831951464
site : 13 1.6346352104967994e-5
site : 14 0.05222041976888438
site : 15 0.0017492709739197476


In [30]:
ψ3 = deepcopy(ψ)
cutoff = 1e-15

ttotal = 0.5
tau = 0.05
current_time = 0
Nsteps = Int(ttotal/tau)

gates = ITensor[]
  for j=1:N-1
    s1 = s[j]
    s2 = s[j+1]
    hj =       op("Sz",s1) * op("Sz",s2) +
         1/2 * op("S+",s1) * op("S-",s2) +
         1/2 * op("S-",s1) * op("S+",s2)
    Gj = exp(-1.0im * tau/2 * hj)
    push!(gates,Gj)
  end


append!(gates,reverse(gates))



for step=1:Nsteps
    psi = apply(gates, ψ3; cutoff=cutoff)
    current_time += tau
    Sz = measure_Sz(ψ3,5)
    #println("$current_time   $Sz")
end


#Calculate magnetisation at all sites.

for j=1:N
    println("site : ",j," ",measure_Sz(ψ3,j))
end



site : 1 0.27902255077953314
site : 2 0.11817057880196195
site : 3 -0.1699914253375467
site : 4 -0.11250046529889368
site : 5 -0.06599371885151037
site : 6 -0.03204545340132578
site : 7 0.05619019903028113
site : 8 0.03549310258727675
site : 9 -0.08882634724034837
site : 10 -0.05604017443275978
site : 11 -0.07076978946509876
site : 12 0.07593901185729657
site : 13 -0.024272228754986722
site : 14 0.07978853217244722
site : 15 0.01457790322514336
